In [ ]:
!python -m pip install findspark
!pip install pyspark

     |█▎                              | 11.2 MB 19 kB/s eta 3:51:006

In [4]:


import os



# ensure SPARK_HOME is correctly set (in .bashrc)
os.environ['SPARK_HOME']



KeyError: 'SPARK_HOME'

In [2]:
# Fonctions pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType, IntegerType, DoubleType, DataType, FloatType
from pyspark.ml.image import ImageSchema
from pyspark.ml.feature import PCA
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector


# Fonction pour ouvrir l'image à partir de son chemin d'accès
from PIL import Image

# Librairies classiques
import numpy as np
import sys
import os
import io
import time

# Librairie pour se connecter au service S3 d'AWS
import boto3


ModuleNotFoundError: No module named 'pyspark'

In [ ]:
def load_datas(folder):
    
    """
    Retourne un dataFrame pyspark avec comme colonne la liste des chemins d'accès
    de toutes les images se trouvant le dossier folder
    """

    # Initialisation du temps de calcul
    start_time = time.time()
    
    lst_path =  []
    
    # Suivant l'argument sys.argv[1], connexion en local ou sur AWS
    if sys.argv[1] == 'True':

        sub_folders = os.listdir(folder)
        print(sub_folders)

        for f in sub_folders:

            lst_categ = os.listdir(folder + f)

            for file in lst_categ:
                lst_path.append(folder + f + "/" + file)
    else :
        # Connexion à l'espace de stockage S3 d'AWS
        session = boto3.session.Session(aws_access_key_id="",
                                        aws_secret_access_key="")
        s3_client = session.client(service_name='s3', region_name="us-east-1")

        prefix = 'data'
        sub_folders = s3_client.list_objects_v2(Bucket="mlgbucket", Prefix=prefix)

        if "Contents" not in sub_folders:
            print("Erreur lors du chargement des images")
            print("Le dossier source n'a pas été trouvé")
            sys.exit(0)

        for key in sub_folders["Contents"]:

            file = key["Key"]
            file = file.replace(prefix + "/", "")
            lst_path.append(folder + file)

    print("Nombre d'images chargées :", len(lst_path))
    # Création d'un RDD à partit de la liste des chemins d'accès aux images
    rdd = sc.parallelize(lst_path)
    row_rdd = rdd.map(lambda x: Row(x))
    # Création d'un dataFrame pyspark à partir d'un RDD
    df = spark.createDataFrame(row_rdd, ["path_img"])

    # Affichage du temps de calcul
    print("Temps d'execution {:.2f} secondes".format(time.time() - start_time))

    return df